**Capítulo 9: Em pleno funcionamento com o TensorFlow**

O Capítulo 9, intitulado "**Up and Running with TensorFlow**" (Em Funcionamento com TensorFlow), serve como uma **introdução essencial à biblioteca TensorFlow**. Esta biblioteca é um **poderoso software de código aberto para computação numérica**, otimizado para **Machine Learning em larga escala**. O principal objetivo deste capítulo é estabelecer os fundamentos e conceitos cruciais do TensorFlow, preparando o leitor para construir redes neurais mais complexas nos capítulos subsequentes.

## 1. A Filosofia do TensorFlow: Grafos de Computação

O TensorFlow opera definindo um **grafo de computação em Python**. Este grafo descreve as operações e o fluxo de dados, mas **não executa nenhum cálculo no momento de sua definição**. Um programa TensorFlow é dividido em duas fases distintas:

*   **Fase de Construção (Construction Phase)**: Nesta etapa, o grafo de computação é construído, representando o modelo de Machine Learning e as operações necessárias para treiná-lo.
*   **Fase de Execução (Execution Phase)**: Nesta fase, o grafo é de fato executado, geralmente em um laço que avalia repetidamente os passos de treinamento, ajustando gradualmente os parâmetros do modelo.

## 2. Executando o Grafo com Sessões

Para avaliar um grafo do TensorFlow, é necessário **abrir uma sessão de TensorFlow**. A sessão é responsável por:

*   **Alocar operações em dispositivos** como CPUs e GPUs.
*   **Executar as operações** definidas no grafo.
*   **Manter os valores de todas as variáveis**.

O capítulo apresenta como usar `tf.Session()` e `tf.InteractiveSession()` para inicializar variáveis e avaliar nós no grafo. Para evitar nós duplicados em ambientes interativos, é recomendado usar `tf.reset_default_graph()`.

## 3. Ciclo de Vida de um Valor de Nó

Quando um nó é avaliado, o TensorFlow **determina e executa automaticamente todos os nós dos quais ele depende**. Isso garante que as operações sejam executadas na ordem correta de dependência. Os inputs e outputs são **tensores**, que são arrays multidimensionais.

## 4. Implementação de Gradiente Descendente

O capítulo detalha a implementação do algoritmo de **Gradiente Descendente**, uma técnica de otimização para minimizar uma função de custo.

*   **Normalização**: É crucial **normalizar os vetores de características de entrada** para acelerar o treinamento.
*   **Gradientes Manuais**: Inicialmente, discute a **computação manual dos gradientes**, um processo tedioso e propenso a erros em redes neurais profundas.
*   **Diferenciação Automática (Autodiff)**: O TensorFlow oferece o recurso de **diferenciação automática** para calcular os gradientes de forma eficiente. O TensorFlow utiliza o **autodiff no modo reverso**, que é altamente eficiente e preciso quando há muitas entradas e poucas saídas, comum em redes neurais. A função `tf.gradients()` é usada para isso.
*   **Otimizadores Pré-prontos**: O TensorFlow fornece otimizadores "out-of-the-box", como `tf.train.GradientDescentOptimizer`, que simplificam ainda mais o processo de treinamento.

## 5. Visualização com TensorBoard

O **TensorBoard** é uma ferramenta essencial para **visualizar e depurar grafos de computação** e monitorar o treinamento do modelo.

*   **Resumos (Summaries)**: É mostrado como adicionar resumos, como `tf.summary.scalar` para monitorar o Erro Quadrático Médio (MSE).
*   **Gravador de Arquivos (FileWriter)**: `tf.summary.FileWriter` é usado para gravar esses resumos e a definição do grafo em arquivos de log compatíveis com o TensorBoard.
*   **Funcionalidades**: O TensorBoard permite visualizar as curvas de treinamento, a estrutura do grafo de computação e é útil para identificar erros e gargalos.

## 6. Modularidade e Escopos de Nomes (Name Scopes)

Para evitar código repetitivo e melhorar a legibilidade do grafo, o capítulo introduz a modularidade.

*   **Funções Auxiliares**: A criação de funções para construir partes do grafo (como uma função `relu()`) ajuda a manter o código mais limpo.
*   **Escopos de Nomes (`tf.name_scope`)**: Permitem agrupar nós relacionados no grafo. Isso torna o grafo mais claro no TensorBoard, que **colapsa séries de nós** com nomes similares para reduzir a desordem. O TensorFlow adiciona sufixos (`_1`, `_2`) para garantir nomes únicos dentro desses escopos.

## 7. Compartilhamento de Variáveis

O capítulo explica como **compartilhar variáveis** entre diferentes partes de um grafo complexo.

*   **`tf.get_variable()` e `tf.variable_scope()`**: Em vez de passar variáveis como parâmetros, o TensorFlow oferece uma maneira mais organizada usando `tf.get_variable()` em conjunto com `tf.variable_scope()`.
*   **Atributo `reuse`**: A função `tf.get_variable()` cria uma variável se ela não existir ou a reutiliza se já existir. O comportamento (criar ou reutilizar) é controlado pelo atributo `reuse` do `tf.variable_scope()` atual. Variáveis criadas com `get_variable()` são nomeadas com o prefixo do `variable_scope`, que também funciona como um `name_scope` para outros nós.

---

### **Implementação:**

In [78]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from sklearn.datasets import make_moons

In [3]:
tf.compat.v1.disable_eager_execution()

graph = tf.Graph()
with graph.as_default():
	x = tf.compat.v1.Variable(3, name='x')
	y = tf.compat.v1.Variable(4, name='y')
	f = x**x * y + y + 2

	sess = tf.compat.v1.Session()
	sess.run(tf.compat.v1.global_variables_initializer())
	result = sess.run(f)
	print(result)


114


In [4]:
sess.close()

In [7]:
sess = tf.compat.v1.InteractiveSession(graph=graph)
sess.run(init)
result = sess.run(f)
print(result)
sess.close()

ERROR:tensorflow:An interactive session is already active. This can cause out-of-memory errors or some other unexpected errors (due to the unpredictable timing of garbage collection) in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s). Please use `tf.Session()` if you intend to productionize.
114


**Gerenciando Grafos**

In [8]:
x1 = tf.Variable(1)
x1.graph is tf.compat.v1.get_default_graph()

True

In [9]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [10]:
x2.graph is graph

True

In [11]:
x2.graph is tf.compat.v1.get_default_graph()

False

**Ciclo de vida do nó**

In [12]:
tf.compat.v1.disable_eager_execution()

w = tf.compat.v1.Variable(3)
x = w + 2
y = x + 5
z = x * 3

with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    print(sess.run(y))
    print(sess.run(z))

10
15


In [13]:
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


Regressão linear com Tensorflow

In [16]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)

In [18]:
with tf.compat.v1.Session() as sess:
    theta_value = sess.run(theta)

Implementando o Gradiente Descendente

In [23]:
n_epochs = 1000
learning_rate = 0.01

scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.compat.v1.assign(theta, theta - learning_rate * gradients)

init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 12.782243
Epoch 100 MSE = 4.9373193
Epoch 200 MSE = 4.8748503
Epoch 300 MSE = 4.855022
Epoch 400 MSE = 4.841163
Epoch 500 MSE = 4.8310885
Epoch 600 MSE = 4.823741
Epoch 700 MSE = 4.8183727
Epoch 800 MSE = 4.814443
Epoch 900 MSE = 4.811562


Utilizando um otimizador

In [27]:
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

Fornecendo Dados ao Algoritmo de Treinamento

In [31]:
A = tf.compat.v1.placeholder(tf.float32, shape=(None, n + 1), name='A')
B = A + 5
with tf.compat.v1.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3, 4, 5, 6, 7, 8, 9]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6, 7, 8, 9, 1, 2, 3], [7, 8, 9, 1, 2, 3, 4, 5, 6]]})
print(B_val_1)
print(B_val_2)

[[ 6.  7.  8.  9. 10. 11. 12. 13. 14.]]
[[ 9. 10. 11. 12. 13. 14.  6.  7.  8.]
 [12. 13. 14.  6.  7.  8.  9. 10. 11.]]


In [33]:
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name='y')

In [34]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [36]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.compat.v1.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()

Salvando e restaurando modelos

In [41]:
theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')

init = tf.compat.v1.global_variables_initializer()
saver = tf.compat.v1.train.Saver()

with tf.compat.v1.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            if epoch % 100 == 0 and batch_index == 0:
                save_path = saver.save(sess, "/tmp/my_model.ckpt")
                print(f"Model saved at epoch {epoch}, batch {batch_index}")

    best_theta = sess.run(theta)
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")
    print("Final model saved.")

Model saved at epoch 0, batch 0
Model saved at epoch 100, batch 0
Model saved at epoch 200, batch 0
Model saved at epoch 300, batch 0
Model saved at epoch 400, batch 0
Model saved at epoch 500, batch 0
Model saved at epoch 600, batch 0
Model saved at epoch 700, batch 0
Model saved at epoch 800, batch 0
Model saved at epoch 900, batch 0
Final model saved.


In [42]:
with tf.compat.v1.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


salvando:

In [46]:
saver = tf.compat.v1.train.Saver({"weights": theta})

restaurando:

In [48]:
saver = tf.compat.v1.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")

with tf.compat.v1.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


Vasualização de Grafo e curvas de treinamento com Tensorboard

In [51]:
now = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

C:\Users\andre\AppData\Local\Temp\ipykernel_13144\3091642509.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y%m%d-%H%M%S")


In [53]:
mse_summary = tf.compat.v1.summary.scalar('MSE', mse)
file_writer = tf.compat.v1.summary.FileWriter(logdir, tf.compat.v1.get_default_graph())

In [55]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for batch_index in range(n_batches):
        X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
        if epoch % 10 == 0:
            summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch}, session=sess)
            file_writer.add_summary(summary_str, epoch * n_batches + batch_index)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

In [56]:
file_writer.close()

Escopos do Nome

In [58]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error, name="mse"))

In [59]:
print(error.op.name)

loss_1/sub


In [61]:
print(mse.op.name)

loss_1/Mean


Modulariadade

In [62]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random.truncated_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.matmul(X, w) + b
    return tf.maximum(z, 0.0, name = "relu")

In [63]:
n_features = 3
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

Compartilhando variáveis

In [66]:
def relu(x):
    with tf.name_scope("relu"):
        w_shape = (int(x.get_shape()[1]), 1)
        w = tf.Variable(tf.random.truncated_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.matmul(x, w) + b
        return tf.maximum(z, 0.0, name = "max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [70]:
with tf.compat.v1.variable_scope("", reuse=tf.compat.v1.AUTO_REUSE):
	threshold = tf.compat.v1.get_variable("threshold", shape=(), initializer=tf.zeros_initializer())

In [76]:
def relu(X):
    with tf.compat.v1.variable_scope("relu", reuse=True):
        threshold = tf.compat.v1.get_variable("threshold", shape=(), initializer=tf.compat.v1.zeros_initializer())
        return tf.maximum(X - threshold, 0.0)
    
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name='X')
with tf.compat.v1.variable_scope("relu", reuse=tf.compat.v1.AUTO_REUSE):
    threshold = tf.compat.v1.get_variable("threshold", shape=(), initializer=tf.compat.v1.zeros_initializer())
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

### **Exercícios**

1. Quais são os principais benefícios de se criar um grafo de cálculo em vez de executar diretamente os cálculos? Quais são as principais desvantagens?

**Benefícios:**
- **Eficiência:** O grafo de cálculo permite otimizações automáticas, como paralelismo e execução em dispositivos especializados (GPUs/TPUs).
- **Portabilidade:** Um grafo pode ser salvo e carregado em diferentes ambientes, facilitando a reutilização e o compartilhamento.
- **Visualização:** Ferramentas como o TensorBoard permitem inspecionar e depurar o grafo, ajudando a entender o fluxo de dados e operações.
- **Modularidade:** O grafo organiza as operações de forma clara, permitindo reutilização de subcomponentes e melhor manutenção do código.
- **Execução Diferida:** A separação entre construção e execução permite definir todo o modelo antes de executá-lo, o que é útil para depuração e planejamento.

**Desvantagens:**
- **Complexidade:** A criação de grafos pode ser mais difícil de entender e implementar, especialmente para iniciantes.
- **Menor Flexibilidade:** A execução diferida pode dificultar a depuração, já que os valores intermediários não estão disponíveis até a execução.
- **Sobrecarga Inicial:** A construção do grafo pode adicionar uma sobrecarga inicial, especialmente para tarefas simples.
- **Curva de Aprendizado:** Requer familiaridade com conceitos como tensores, sessões e escopos, o que pode ser desafiador para novos usuários.
```

2. A declaração "a_val = a.eval(session=sess)" é equivalente a "a_val = sess.run(a)"?

Sim, a declaração `a_val = a.eval(session=sess)` é equivalente a `a_val = sess.run(a)` no TensorFlow 1.x. Ambas executam o nó `a` no grafo de computação dentro da sessão `sess` e retornam o valor resultante. A diferença é apenas sintática:
- `a.eval(session=sess)` é um método do próprio tensor, onde você especifica a sessão.
- `sess.run(a)` é um método da sessão, onde você especifica o tensor a ser avaliado.
Ambas as formas são válidas e produzem o mesmo resultado.

3. A declaração a_val, b_val = a.val(session=sess), b.eval(session=sess) é equivalente a a_val, b_val = sess.run([a,b])?

Sim, a declaração `a_val, b_val = a.eval(session=sess), b.eval(session=sess)` é funcionalmente equivalente a `a_val, b_val = sess.run([a, b])` no TensorFlow 1.x. Ambas avaliam os tensores `a` e `b` na mesma sessão e retornam seus valores.
No entanto, usar `sess.run([a, b])` é mais eficiente, pois ambos os tensores são avaliados em uma única execução do grafo, aproveitando o compartilhamento de operações intermediárias. Avaliar separadamente pode resultar em cálculos duplicados se `a` e `b` compartilharem partes do grafo.

4. É possível executar dois grafos na mesma sessão?

Não, no TensorFlow 1.x, uma sessão (`tf.Session`) está associada a um único grafo de computação. Se você tentar executar operações de dois grafos diferentes na mesma sessão, ocorrerá um erro. No entanto, você pode criar múltiplas sessões, cada uma associada a um grafo diferente, ou usar o método `with graph.as_default()` para alternar entre grafos no mesmo código.


5. Se você criar um grafo g contendo uma variável w, então iniciar dois segmentos e abrir uma sessão em cada, ambos utilizando o mesmo grafo g, cada sessão terá sua própria cópia da variável w ou esta será compartilhada?

Cada sessão terá sua própria cópia da variável `w`. No TensorFlow 1.x, as variáveis são armazenadas no contexto da sessão, e cada sessão mantém seu próprio estado das variáveis. Isso significa que, mesmo que as sessões compartilhem o mesmo grafo, as variáveis não são compartilhadas entre elas. Cada sessão inicializa e gerencia suas próprias instâncias das variáveis.

6. Quando uma variável é iniciada? Quando é destruída?

**Início:**
Uma variável no TensorFlow 1.x é iniciada explicitamente ao executar a operação de inicialização, como `sess.run(tf.global_variables_initializer())`, ou ao executar uma operação que depende dela. Até que seja inicializada, a variável não possui valor.

**Destruição:**
Uma variável é destruída quando a sessão associada a ela é fechada. Como as variáveis são armazenadas no contexto da sessão, fechar a sessão (`sess.close()`) libera os recursos alocados para essas variáveis.

7. Qual é a diferença entre um placeholder e uma variável?

**Placeholder:**
- Um placeholder é um nó no grafo que serve como entrada para os dados. Ele não armazena nenhum valor por si só.
- É usado para alimentar dados no grafo durante a execução, utilizando o argumento `feed_dict` em uma sessão.
- Deve ter seu valor fornecido no momento da execução, caso contrário, ocorrerá um erro.
- Exemplo: `X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_features), name='X')`

**Variável:**
- Uma variável é um nó no grafo que armazena um valor persistente, que pode ser atualizado durante a execução.
- É usada para armazenar pesos, vieses ou outros parâmetros do modelo.
- Deve ser inicializada explicitamente antes de ser usada.
- Exemplo: `theta = tf.Variable(tf.random.uniform([n + 1, 1], -1.0, 1.0), name='theta')`

**Resumo:**
- Placeholders são usados para fornecer dados ao grafo, enquanto variáveis são usadas para armazenar e atualizar valores durante o treinamento.

8. O que acontece quando você executa o grafo para avaliar uma operação que depende de um placeholder, mas você não fornece seu valor? O que acontece se a operação não depender do placeholder?

- **Se a operação depender do placeholder:** O TensorFlow lançará um erro (`InvalidArgumentError`) porque o valor do placeholder não foi fornecido. Placeholders exigem que seus valores sejam passados por meio do argumento `feed_dict` durante a execução.

- **Se a operação não depender do placeholder:** A operação será avaliada normalmente, pois o placeholder não é necessário para calcular o resultado. Nesse caso, o TensorFlow ignora o placeholder durante a execução.

9. Quando você executa um grafo, você pode fornecer o valor de saída de qualquer operação ou apenas o valor dos placeholders?

Quando você executa um grafo no TensorFlow 1.x, você pode fornecer valores apenas para os placeholders. Esses valores são passados por meio do argumento `feed_dict` na chamada de execução da sessão (`sess.run()`).

As operações no grafo, por outro lado, não podem ter seus valores fornecidos diretamente. Elas são calculadas automaticamente com base nos valores dos placeholders e nas dependências definidas no grafo. O TensorFlow avalia as operações necessárias para produzir o resultado solicitado, seguindo o fluxo de dependências do grafo.

10. Como você pode definir uma variável para qualquer valor desejado (durante a fase de execução)?

Para definir uma variável para qualquer valor desejado durante a fase de execução no TensorFlow 1.x, você pode usar o método `assign()` ou o método `feed_dict` com um placeholder associado à variável. Aqui está como fazer isso:

1. **Usando `assign`:**
    ```python
    assign_op = variable.assign(new_value)
    sess.run(assign_op)
    ```

2. **Usando um placeholder:**
    ```python
    placeholder = tf.compat.v1.placeholder(dtype=tf.float32, shape=variable.shape)
    assign_op = variable.assign(placeholder)
    sess.run(assign_op, feed_dict={placeholder: new_value})
    ```

Ambos os métodos permitem atualizar o valor da variável durante a execução do grafo.

11. Quantas vezes o autodiff de modo reverso precisa percorrer o gráfico para calcular os gradientes da função de custo em relação a dez variáveis? E quanto ao autodiff do modo avançado? E a diferenciação simbólica?

- **Autodiff de modo reverso:** O autodiff de modo reverso percorre o gráfico **uma única vez** para calcular os gradientes da função de custo em relação a todas as dez variáveis. Isso ocorre porque ele acumula os gradientes de forma eficiente durante a retropropagação.

- **Autodiff do modo avançado:** O autodiff do modo avançado precisaria percorrer o gráfico **dez vezes**, uma vez para cada variável, porque ele calcula os gradientes de forma independente para cada variável.

- **Diferenciação simbólica:** A diferenciação simbólica não percorre o gráfico, mas gera expressões analíticas para os gradientes. No entanto, essas expressões podem ser computacionalmente caras e propensas a explosão de complexidade para gráficos grandes.

12. Implemente a Regressão Logística com Gradiente Descendente utilizando Minilotes no TensorFlow. Treine e avalie o modelo no conjunto de dados em formato de luas. Adicione as seguintes funcionalidades:

**A. Definição do Grafo**
- Crie o grafo dentro da função `logistic_regression()`, permitindo reutilização fácil.

**B. Salvamento de Pontos de Verificação**
- Utilize um `Saver` para salvar pontos de verificação em intervalos regulares durante o treinamento.
- Salve o modelo final ao término do treinamento.

**C. Restauração de Pontos de Verificação**
- Restaure o último ponto de verificação automaticamente após a inicialização, caso o treinamento seja interrompido.

**D. Escopos de Nome**
- Defina o grafo utilizando escopos de nome (`name_scope`) para melhorar a visualização no TensorBoard.

**E. Resumos para o TensorBoard**
- Adicione resumos (`summaries`) para visualizar as curvas de aprendizado no TensorBoard.

**F. Ajuste de Hiperparâmetros**
- Experimente ajustar hiperparâmetros, como:
    - Taxa de aprendizado (`learning_rate`)
    - Tamanho do minilote (`batch_size`)
- Observe como essas alterações afetam a curva de aprendizado.